# Symmetric/bosonic extension

In [ ]:
import numpy as np
import numqi

Symmetric extension is a hierarchical method to approximate the set of separable states. A bipartite state $\rho_{AB}$ is said to be $k$-symmetric extendible if there exists a global state $\rho_{AB^k}:=\rho_{AB_1B_2...B_k}$ such that for any $i,j$

$$
\begin{cases}
\operatorname{Tr}_{B_2 \cdots B_k}\left[\rho_{A B^k}\right] & =\rho_{A B}, \\
\left(\mathbb{I}_A \otimes P_{i j}\right) \rho_{A B^k}\left(\mathbb{I}_A \otimes P_{i j}\right) & =\rho_{A B^k},
\end{cases}
$$

where $P_{ij}$ is the permutation operator that swaps the $i$-th and $j$-th subsystems. The set of $k$-symmetric extendible states is denoted by $\Theta_{k}$, which is convex with a hierarchical structure that $\Theta_{k+1} \subset \Theta_{k}$. Moreover, when $k \rightarrow \infty$, $\Theta_{k}$ converges to the set of separable states.

The $k$-bosonic extendible set $\bar{\Theta}_k$ is a convex subset of $\Theta_k$, which further requires the global state $\rho_{AB^k}$ is supported on the symmetric subspace of $B_1B_2...B_k$. Similarly, we have the hierarchical relation $\bar{\Theta}_{k+1} \subset \bar{\Theta}_{k}$ and $\bar{\Theta}_{\infty} = SEP$. PPT constraint can also be considered.

In summary, given a quantum state $\rho_{AB}$, we can use the following semidefinite program to check whether it is (PPT) $k$-symmetric/bosonic extendible:

$$
\text{Find}~\rho_{AB^k}
$$

$$
s.t.\;
\begin{cases}
\operatorname{Tr}_{B_2 \cdots B_k}\left[\rho_{A B^k}\right] & =\rho_{A B}, \\
\rho_{AB^k} \succeq 0,\\
\left(\mathbb{I}_A \otimes P_{i j}\right) \rho_{A B^k}\left(\mathbb{I}_A \otimes P_{i j}\right) & =\rho_{A B^k}, ~(\text{Symmetric})\\
\left(\mathbb{I}_A \otimes P_{i j}\right) \rho_{A B^k} & =\rho_{A B^k}, ~(\text{Bosonic})\\
\rho_{AB^k}^{\Gamma_A} \succeq 0, ~(\text{PPT})
\end{cases}
$$


In [ ]:
dimA = 3
dimB = 3
kext = 2
# generate a random separable state
rho_AB = numqi.random.rand_separable_dm(dimA, dimB)
# check if the state is k-symmetric extendible
is_kext = numqi.entangle.check_ABk_symmetric_extension(rho_AB, (dimA,dimB), kext, use_ppt=False, use_boson=False)
print("Is the state k-symmetric extendible?", is_kext)

Since the (PPT) $k$-symmetric/bosonic extendible set converges to the set of separable states with $k$ increases, we can use the following semidefinite program to approximate the relative entropy of entanglement (REE) for a given state $\rho_{AB}$, which is a lower bound of the REE:

$$ \min_{\sigma_{A B}} S(\sigma_{A B} \| \rho_{AB})$$

$$
s.t.\;
\begin{cases}
\sigma_{AB^k} \succeq 0, ~\operatorname{Tr}[\sigma_{AB^k}]=1,\\
\operatorname{Tr}_{B_2 \cdots B_k}\left[\sigma_{A B^k}\right] & =\sigma_{A B}, \\
\left(\mathbb{I}_A \otimes P_{i j}\right) \sigma_{A B^k}\left(\mathbb{I}_A \otimes P_{i j}\right) & =\sigma_{A B^k}, ~(\text{Symmetric})\\
\left(\mathbb{I}_A \otimes P_{i j}\right) \sigma_{A B^k} & =\sigma_{A B^k}, ~(\text{Bosonic})\\
\sigma_{AB^k}^{\Gamma_A} \succeq 0, ~(\text{PPT})
\end{cases}
$$

In [ ]:
dimA = 3
dimB = 3
kext = 2
# generate a random separable state
rho_AB = numqi.random.rand_separable_dm(dimA, dimB)
# approximate ree
ree = numqi.entangle.get_ABk_symmetric_extension_ree(rho_AB, (dimA,dimB), kext, use_ppt=False, use_boson=False)
print("approximated REE of the random separable state:", ree)

Take the Werner state as an example, which has the following form

$$ \rho_{W,d}\left(\alpha\right)=\frac{1}{d^{2}-d\alpha}I-\frac{\alpha}{d^{2}-d\alpha}\sum_{ij}\left|ij\right\rangle \left\langle ji\right|,\quad\alpha\in\left[-1,1\right] $$

Werner state is separable for $\alpha\in\left[-1,\frac{1}{d}\right]$, and entangled for $\alpha\in\left(\frac{1}{d},1\right]$. The REE of Werner state is given in paper "Entanglement measures under symmetry" [doi-link](https://doi.org/10.1103/PhysRevA.64.062307).

$$ \mathrm{REE\left(\rho_{W,d}(\alpha)\right)=\begin{cases}
0 & \alpha\in\left[-1,\frac{1}{d}\right]\\
S\left(\rho_{W,d}(\alpha)\lvert\rho_{W,d}\left(\frac{1}{d}\right)\right) & \alpha\in\left[\frac{1}{d},1\right]
\end{cases}} $$

In [ ]:
dim = 2
alpha_list = np.linspace(0, 1, 100, endpoint=False) # alpha=1 is unstable for matrix logarithm
dm_target_list = [numqi.state.Werner(dim, x) for x in alpha_list]
# Analytical REE of Werner states
ree_analytical = np.array([numqi.state.get_Werner_ree(dim, x) for x in alpha_list])
# REE of Werner states obtained by PPT
ree_ppt = numqi.entangle.get_ppt_ree(dm_target_list, dim, dim)
kext = [2, 4, 6]
# Create an two-dimensional array to store the REE of Werner states obtained by symmetric extension
ree_sym_ext = []
for k in kext:
    ree_kext = []
    for dm_target in dm_target_list:
        ree_kext.append(numqi.entangle.get_ABk_symmetric_extension_ree(dm_target, (dim,dim), k, use_ppt=False, use_boson=False))
    ree_sym_ext.append(ree_kext)
ree_sym_ext = np.array(ree_sym_ext)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(alpha_list, ree_analytical, "x", label="analytical")
ax.plot(alpha_list, ree_ppt, label="PPT")
for k, ree_kext in zip(kext, ree_sym_ext):
    ax.plot(alpha_list, ree_kext, label=f"{k}-symmetric extension")
ax.set_xlim(min(alpha_list), max(alpha_list))
ax.set_ylim(1e-13, 1)
ax.set_yscale('log')
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel("REE")
ax.set_title(f'Werner({dim})')
ax.legend()
fig.tight_layout()